<a href="https://colab.research.google.com/github/LatiefDataVisionary/demand-forecasting-retail-college-task/blob/main/notebooks/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demand Forecasting for Retail: A Machine Learning Pipeline Approach**

## **Project Overview**

**College Course:** Model Development Engineering

**Objective:** To develop a robust machine learning model to accurately forecast daily product demand for a retail company. This involves a comprehensive data pipeline, including merging multiple data sources, extensive feature engineering, and model comparison to optimize inventory management and reduce costs.

## **Chapter 1: Project Setup**

This chapter covers the initial setup, including importing necessary libraries and loading the datasets from their sources.

### **1.1. Import Libraries**

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb

# Set plotting style
sns.set_style("whitegrid")

### **1.2. Load Datasets**

In [22]:
data1 = 'https://raw.githubusercontent.com/LatiefDataVisionary/demand-forecasting-retail-college-task/refs/heads/main/data/raw/Dataset%201_Retail%20Store%20Inventory%20and%20Demand%20Forecasting.csv'
data2 = 'https://raw.githubusercontent.com/LatiefDataVisionary/demand-forecasting-retail-college-task/refs/heads/main/data/raw/Dataset%202_Retail%20Sales%20Data%20with%20Seasonal%20Trends%20%26%20Marketing.csv'
data3 = 'https://raw.githubusercontent.com/LatiefDataVisionary/demand-forecasting-retail-college-task/refs/heads/main/data/raw/Dataset%203_Strategic%20Supply%20Chain%20Demand%20Forecasting%20Dataset.csv'

df1 = pd.read_csv(data1)
df2 = pd.read_csv(data2)
df3 = pd.read_csv(data3)

print("df1 loaded successfully.")
print("df2 loaded successfully.")
print("df3 loaded successfully.")

df1 loaded successfully.
df2 loaded successfully.
df3 loaded successfully.


#### **1.2.1. Data Information**

In [23]:
df1.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Price,Discount,Weather Condition,Promotion,Competitor Pricing,Seasonality,Epidemic,Demand
0,2022-01-01,S001,P0001,Electronics,North,195,102,252,72.72,5,Snowy,0,85.73,Winter,0,115
1,2022-01-01,S001,P0002,Clothing,North,117,117,249,80.16,15,Snowy,1,92.02,Winter,0,229
2,2022-01-01,S001,P0003,Clothing,North,247,114,612,62.94,10,Snowy,1,60.08,Winter,0,157
3,2022-01-01,S001,P0004,Electronics,North,139,45,102,87.63,10,Snowy,0,85.19,Winter,0,52
4,2022-01-01,S001,P0005,Groceries,North,152,65,271,54.41,0,Snowy,0,51.63,Winter,0,59


In [26]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76000 entries, 0 to 75999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                76000 non-null  object 
 1   Store ID            76000 non-null  object 
 2   Product ID          76000 non-null  object 
 3   Category            76000 non-null  object 
 4   Region              76000 non-null  object 
 5   Inventory Level     76000 non-null  int64  
 6   Units Sold          76000 non-null  int64  
 7   Units Ordered       76000 non-null  int64  
 8   Price               76000 non-null  float64
 9   Discount            76000 non-null  int64  
 10  Weather Condition   76000 non-null  object 
 11  Promotion           76000 non-null  int64  
 12  Competitor Pricing  76000 non-null  float64
 13  Seasonality         76000 non-null  object 
 14  Epidemic            76000 non-null  int64  
 15  Demand              76000 non-null  int64  
dtypes: f

In [27]:
df2.head()

,YEAR,MONTH,SUPPLIER,ITEM CODE,ITEM DESCRIPTION,ITEM TYPE,RETAIL SALES,RETAIL TRANSFERS,WAREHOUSE SALES
0,2020,1,REPUBLIC NATIONAL DISTRIBUTING CO,100009,BOOTLEG RED - 750ML,WINE,0.00,0.0,2.0
1,2020,1,PWSWN INC,100024,MOMENT DE PLAISIR - 750ML,WINE,0.00,1.0,4.0
2,2020,1,RELIABLE CHURCHILL LLLP,1001,S SMITH ORGANIC PEAR CIDER - 18.7OZ,BEER,0.00,0.0,1.0
3,2020,1,LANTERNA DISTRIBUTORS INC,100145,SCHLINK HAUS KABINETT - 750ML,WINE,0.00,0.0,1.0
4,2020,1,DIONYSOS IMPORTS INC,100293,SANTORINI GAVALA WHITE - 750ML,WINE,0.82,0.0,0.0


In [28]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   YEAR              30000 non-null  int64  
 1   MONTH             30000 non-null  int64  
 2   SUPPLIER          29967 non-null  object 
 3   ITEM CODE         30000 non-null  object 
 4   ITEM DESCRIPTION  30000 non-null  object 
 5   ITEM TYPE         30000 non-null  object 
 6   RETAIL SALES      29999 non-null  float64
 7   RETAIL TRANSFERS  30000 non-null  float64
 8   WAREHOUSE SALES   30000 non-null  float64
dtypes: float64(3), int64(2), object(4)
memory usage: 2.1+ MB


In [29]:
df3.head()

,date,product_id,sales_units,holiday_season,promotion_applied,competitor_price_index,economic_index,weather_impact,price,discount_percentage,sales_revenue,region_Europe,region_North America,store_type_Retail,store_type_Wholesale,category_Cabinets,category_Chairs,category_Sofas,category_Tables,future_demand
0,2023-01-01,151,99,0,0,0.983893,1.314333,0,126.932922,0.000000,12566.359326,False,False,True,False,False,True,False,False,95.0
1,2023-01-02,192,95,1,0,0.977615,1.439582,0,151.355405,0.000000,14378.763512,False,True,False,False,True,False,False,False,101.0
2,2023-01-03,114,101,0,0,0.983913,1.094795,0,191.701693,0.000000,19361.870952,False,False,False,True,True,False,False,False,33.0
3,2023-01-04,171,33,0,0,1.191956,0.907672,0,173.106487,0.000000,5712.514059,False,False,False,True,False,True,False,False,82.0
4,2023-01-05,160,82,0,1,0.855711,1.479690,0,138.587491,10.995213,10114.659145,True,False,False,False,False,False,True,False,13.0


In [30]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    4999 non-null   object 
 1   product_id              4999 non-null   int64  
 2   sales_units             4999 non-null   int64  
 3   holiday_season          4999 non-null   int64  
 4   promotion_applied       4999 non-null   int64  
 5   competitor_price_index  4999 non-null   float64
 6   economic_index          4999 non-null   float64
 7   weather_impact          4999 non-null   int64  
 8   price                   4999 non-null   float64
 9   discount_percentage     4999 non-null   float64
 10  sales_revenue           4999 non-null   float64
 11  region_Europe           4999 non-null   bool   
 12  region_North America    4999 non-null   bool   
 13  store_type_Retail       4999 non-null   bool   
 14  store_type_Wholesale    4999 non-null   

#### **1.2.2. Data Summary Statistics**

#### **1.2.3. Unique Values and Counts for Object Columns**

#### **1.2.4. Missing Values Analysis**

#### **1.2.5. Duplicate Values Analysis**

## Chapter 2: Data Ingestion and Merging

The first step in our pipeline is to integrate the three disparate datasets into a single, unified master DataFrame that will serve as the foundation for our analysis and modeling.

### 2.1. Prepare Datasets for Merging

Standardize column names for consistency before merging.

In [ ]:
# Rename columns in df2 and df3
df2 = df2.rename(columns={'ITEM CODE': 'Product ID'})
df3 = df3.rename(columns={'date': 'Date', 'product_id': 'Product ID'})

# Convert Product ID in df2 to string to match df1 and df3
df2['Product ID'] = df2['Product ID'].astype(str)
df1['Product ID'] = df1['Product ID'].astype(str)
df3['Product ID'] = df3['Product ID'].astype(str)


print("Column names standardized and Product ID data types converted.")

### 2.2. Merge Datasets

Perform a two-step merge process to create the `master_df`.

In [ ]:
# Merge df1 with df2
master_df = pd.merge(df1, df2[['Product ID', 'SUPPLIER', 'ITEM TYPE']], on='Product ID', how='left')

# Merge the result with df3
master_df = pd.merge(master_df, df3[['Date', 'Product ID', 'holiday_season', 'promotion_applied',
                                     'competitor_price_index', 'economic_index', 'weather_impact',
                                     'price', 'discount_percentage', 'sales_revenue', 'region_Europe',
                                     'region_North America', 'store_type_Retail', 'store_type_Wholesale',
                                     'category_Cabinets', 'category_Chairs', 'category_Sofas',
                                     'category_Tables', 'future_demand']],
                     on=['Date', 'Product ID'], how='left')

print("Datasets merged successfully.")
display(master_df.head())

## Chapter 3: The 10-Step Data Preparation Pipeline

This chapter details the comprehensive, 10-step data preparation and cleaning process required to transform the raw, merged data into a feature-rich, model-ready format.

### Step 1: Data Cleaning and Type Conversion

Inspect the Master DataFrame and handle missing values.

In [ ]:
# Inspect the Master DataFrame
print("Info of master_df:")
master_df.info()

print("\nHead of master_df:")
display(master_df.head())

# Handle Missing Values
print("\nMissing values before handling:")
print(master_df.isnull().sum().to_markdown(numalign="left", stralign="left"))

# Fill missing 'SUPPLIER' with "Unknown"
master_df['SUPPLIER'] = master_df['SUPPLIER'].fillna('Unknown')

# Fill missing numerical columns with the median
numerical_cols_with_missing = ['RETAIL SALES', 'holiday_season', 'promotion_applied',
                               'competitor_price_index', 'economic_index', 'weather_impact',
                               'price', 'discount_percentage', 'sales_revenue', 'future_demand']

for col in numerical_cols_with_missing:
    if master_df[col].isnull().any():
        median_val = master_df[col].median()
        master_df[col] = master_df[col].fillna(median_val)
        print(f"Filled missing values in {col} with median ({median_val}).")


print("\nMissing values after handling:")
print(master_df.isnull().sum().to_markdown(numalign="left", stralign="left"))

# Convert Date column to datetime object
master_df['Date'] = pd.to_datetime(master_df['Date'])
print("\n'Date' column converted to datetime.")

# Check for Duplicates
print("\nDuplicate rows in master_df:")
print(master_df.duplicated().sum())

### Step 2: Column Consolidation and Selection

Identify and drop redundant or irrelevant columns. Select the target variable.

In [ ]:
# Identify and Drop Redundant Columns
# 'future_demand' is a data leak and must be dropped.
# 'ITEM DESCRIPTION' might be redundant given 'Product ID'.
columns_to_drop = ['future_demand', 'ITEM DESCRIPTION']
master_df = master_df.drop(columns=columns_to_drop)

print(f"Dropped redundant columns: {columns_to_drop}")

# Target Variable Selection
target_variable = 'Demand'
y = master_df[target_variable]
X = master_df.drop(columns=[target_variable])

print(f"Target variable '{target_variable}' selected.")
print("Features DataFrame (X) created.")

### Step 3: Exploratory Data Analysis (EDA)

Visualize key aspects of the data to understand trends, distributions, and relationships.

In [ ]:
# Time Series Analysis: Plot daily average Demand
daily_demand = master_df.groupby('Date')['Demand'].mean().reset_index()

plt.figure(figsize=(15, 6))
sns.lineplot(data=daily_demand, x='Date', y='Demand')
plt.title('Daily Average Demand Over Time')
plt.xlabel('Date')
plt.ylabel('Average Demand')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Distribution Analysis
# Histograms for numerical features
numerical_features = ['Price', 'Inventory Level', 'Units Sold', 'Demand', 'RETAIL SALES',
                      'competitor_price_index', 'economic_index', 'price', 'discount_percentage',
                      'sales_revenue']

master_df[numerical_features].hist(figsize=(15, 10), bins=30)
plt.suptitle('Histograms of Numerical Features', y=1.02)
plt.tight_layout()
plt.show()

# Count plots for categorical features
categorical_features = ['Category', 'Region', 'Store ID', 'Weather Condition', 'Seasonality', 'ITEM TYPE']

for col in categorical_features:
    plt.figure(figsize=(10, 5))
    sns.countplot(data=master_df, y=col, order=master_df[col].value_counts().index)
    plt.title(f'Count Plot of {col}')
    plt.xlabel('Count')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

# Correlation Analysis for numerical features
plt.figure(figsize=(12, 10))
correlation_matrix = master_df[numerical_features].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Numerical Features')
plt.tight_layout()
plt.show()

### Step 4: Outlier Detection and Treatment

Visualize potential outliers and provide an example of outlier treatment.

In [ ]:
# Visualize Outliers using Box Plots
outlier_features = ['Units Sold', 'Price', 'Demand', 'Inventory Level', 'RETAIL SALES']

plt.figure(figsize=(15, 8))
master_df[outlier_features].boxplot()
plt.title('Box Plots for Outlier Detection')
plt.ylabel('Value')
plt.tight_layout()
plt.show()

# Treat Outliers (Optional but Recommended)
# Explain the concept of capping outliers using the IQR method.
print("Outlier Treatment (Capping using IQR method):")
print("The Interquartile Range (IQR) method can be used to identify outliers.")
print("Values below Q1 - 1.5*IQR or above Q3 + 1.5*IQR can be considered outliers.")
print("Capping involves replacing these outliers with the boundary values (Q1 - 1.5*IQR or Q3 + 1.5*IQR).")
print("Below is commented-out code demonstrating how to cap outliers for 'Units Sold' as an example:")

# Example commented-out code for capping 'Units Sold' outliers:
# Q1 = master_df['Units Sold'].quantile(0.25)
# Q3 = master_df['Units Sold'].quantile(0.75)
# IQR = Q3 - Q1
# lower_bound = Q1 - 1.5 * IQR
# upper_bound = Q3 + 1.5 * IQR
# master_df['Units Sold_capped'] = master_df['Units Sold'].clip(lower=lower_bound, upper=upper_bound)
# print("\nExample: 'Units Sold' capped (new column 'Units Sold_capped' created).")

### Step 5: Feature Engineering I - Time-Based Features

Create new features extracted from the 'Date' column.

In [ ]:
# Create time-based features
master_df['year'] = master_df['Date'].dt.year
master_df['month'] = master_df['Date'].dt.month
master_df['day'] = master_df['Date'].dt.day
master_df['dayofweek'] = master_df['Date'].dt.dayofweek # Monday=0, Sunday=6
master_df['dayofyear'] = master_df['Date'].dt.dayofyear
master_df['weekofyear'] = master_df['Date'].dt.isocalendar().week.astype(int)
master_df['quarter'] = master_df['Date'].dt.quarter

print("Time-based features created.")
display(master_df[['Date', 'year', 'month', 'day', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter']].head())

### Step 6: Feature Engineering II - Lag Features

Lag features are crucial for time-series forecasting as they represent past values of the target variable, capturing temporal dependencies.

In [ ]:
# Create lag features for the 'Demand' column
master_df['demand_lag_7'] = master_df.groupby('Product ID')['Demand'].shift(7)
master_df['demand_lag_28'] = master_df.groupby('Product ID')['Demand'].shift(28)

# Handle resulting NaN values (e.g., fill with 0 or the mean of the lag feature)
# Filling with 0 as these are the initial periods where no lag data is available
master_df['demand_lag_7'] = master_df['demand_lag_7'].fillna(0)
master_df['demand_lag_28'] = master_df['demand_lag_28'].fillna(0)


print("Lag features created for 'Demand'.")
display(master_df[['Date', 'Product ID', 'Demand', 'demand_lag_7', 'demand_lag_28']].head(10))

### Step 7: Feature Engineering III - Rolling Window Features

Rolling window features, such as rolling means, help smooth out noise and capture short-term trends in the data.

In [ ]:
# Create rolling mean features for 'Demand'
master_df['demand_rolling_mean_7'] = master_df.groupby('Product ID')['Demand'].transform(lambda x: x.rolling(window=7).mean())
master_df['demand_rolling_mean_28'] = master_df.groupby('Product ID')['Demand'].transform(lambda x: x.rolling(window=28).mean())

# Handle resulting NaN values (initial periods)
# Filling with the mean of the rolling window feature after calculation
master_df['demand_rolling_mean_7'] = master_df['demand_rolling_mean_7'].fillna(master_df['demand_rolling_mean_7'].mean())
master_df['demand_rolling_mean_28'] = master_df['demand_rolling_mean_28'].fillna(master_df['demand_rolling_mean_28'].mean())


print("Rolling window features created for 'Demand'.")
display(master_df[['Date', 'Product ID', 'Demand', 'demand_rolling_mean_7', 'demand_rolling_mean_28']].head(30)) # Display more rows to see non-NaN values

### Step 8: Categorical Feature Encoding

Convert categorical features into a numerical format using One-Hot Encoding.

In [ ]:
# Identify categorical columns (excluding 'Date' and 'Product ID' which are not features for encoding here)
categorical_features_for_encoding = master_df.select_dtypes(include='object').columns.tolist()
# Remove columns that are not intended for one-hot encoding or have been handled
categorical_features_for_encoding.remove('Store ID') # Assuming Store ID will be used as is or dropped later if not needed
# Note: 'Category', 'Region', 'Weather Condition', 'Seasonality', 'SUPPLIER', 'ITEM TYPE' are good candidates

print(f"Categorical features to encode: {categorical_features_for_encoding}")

# Apply One-Hot Encoding
master_df = pd.get_dummies(master_df, columns=categorical_features_for_encoding, dummy_na=False)

print("Categorical features encoded using One-Hot Encoding.")
display(master_df.head())

### Step 9: Feature Scaling

Scale numerical features to standardize their range, which is important for many machine learning models.

In [ ]:
# Identify numerical features to scale
# Exclude 'Date', 'Product ID', the target variable 'Demand', and the newly created binary/boolean columns from one-hot encoding
numerical_features_to_scale = X.select_dtypes(include=np.number).columns.tolist()

# Remove boolean columns resulting from one-hot encoding if they are still in X
boolean_cols = X.select_dtypes(include='bool').columns.tolist()
numerical_features_to_scale = [col for col in numerical_features_to_scale if col not in boolean_cols]

# Exclude lag and rolling features as they are already derived from scaled data or handled
lag_rolling_features = ['demand_lag_7', 'demand_lag_28', 'demand_rolling_mean_7', 'demand_rolling_mean_28']
numerical_features_to_scale = [col for col in numerical_features_to_scale if col not in lag_rolling_features]

print(f"Numerical features to scale: {numerical_features_to_scale}")

# Apply StandardScaler
scaler = StandardScaler()
master_df[numerical_features_to_scale] = scaler.fit_transform(master_df[numerical_features_to_scale])

print("Numerical features scaled using StandardScaler.")
display(master_df.head())

### Step 10: Time-Based Data Splitting

For time-series data, it is critical to split the data chronologically to avoid data leakage from the future into the training set.

In [ ]:
# Define the chronological cutoff date for splitting
# Using a date in early 2023 as an example cutoff
cutoff_date = pd.to_datetime('2023-01-01')

# Split data based on the cutoff date
train_df = master_df[master_df['Date'] < cutoff_date].copy()
val_df = master_df[master_df['Date'] >= cutoff_date].copy()

# Define features (X) and target (y) for train and validation sets
# Exclude 'Date' and 'Product ID' as they are not features for the model
features = [col for col in master_df.columns if col not in ['Date', 'Product ID', target_variable]]

X_train = train_df[features]
y_train = train_df[target_variable]

X_val = val_df[features]
y_val = val_df[target_variable]


print(f"Data split into training and validation sets based on cutoff date: {cutoff_date}")
print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")

## Chapter 4: Model Training and Evaluation

This chapter focuses on selecting, training, and evaluating a machine learning model for demand forecasting.

### 4.1. Model Selection

We will use XGBoost (Extreme Gradient Boosting) for our demand forecasting model due to its strong performance on structured data and its ability to handle various types of features.

### 4.2. Model Training

In [ ]:
# Initialize and train the XGBoost Regressor model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', # Regression task
                             n_estimators=1000,          # Number of boosting rounds
                             learning_rate=0.05,         # Step size shrinkage
                             max_depth=7,                # Maximum depth of a tree
                             random_state=42,            # For reproducibility
                             n_jobs=-1)                  # Use all available cores

print("Training XGBoost model...")
xgb_model.fit(X_train, y_train)
print("XGBoost model training complete.")

### 4.3. Model Evaluation

Evaluate the trained model using appropriate regression metrics on the validation set.

In [ ]:
# Make predictions on the validation set
y_pred = xgb_model.predict(X_val)

# Evaluate the model
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
mae = mean_absolute_error(y_val, y_pred)

print(f"Model Evaluation on Validation Set:")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")

### 4.4. Feature Importance

Analyze the importance of different features in the trained model.

In [ ]:
# Get feature importances from the trained model
feature_importances = xgb_model.feature_importances_

# Create a DataFrame of feature importances
feature_importance_df = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances})

# Sort features by importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Display top N features
print("\nTop 15 Feature Importances:")
display(feature_importance_df.head(15))

# Plot feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance_df.head(15))
plt.title('Top 15 Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

## Chapter 5: Model Evaluation

In this chapter, we evaluate the performance of our trained models on the unseen validation set.

### 5.1. Make Predictions

In [ ]:
# Make predictions with all three models on X_val
y_pred_xgb = xgb_model.predict(X_val)
# Assuming LinearRegression and RandomForestRegressor models were trained in Chapter 4
# If not, you would need to train them here first
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor

# lr_model = LinearRegression()
# lr_model.fit(X_train, y_train)
# y_pred_lr = lr_model.predict(X_val)

# rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)
# y_pred_rf = rf_model.predict(X_val)

print("Predictions made with XGBoost model.")
# print("Predictions made with Linear Regression model.")
# print("Predictions made with Random Forest Regressor model.")

### 5.2. Compare Performance

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Calculate RMSE and MAE for each model
rmse_xgb = np.sqrt(mean_squared_error(y_val, y_pred_xgb))
mae_xgb = mean_absolute_error(y_val, y_pred_xgb)

# Assuming y_pred_lr and y_pred_rf are available from the previous cell
# rmse_lr = np.sqrt(mean_squared_error(y_val, y_pred_lr))
# mae_lr = mean_absolute_error(y_val, y_pred_lr)

# rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf))
# mae_rf = mean_absolute_error(y_val, y_pred_rf)


# Present the results clearly in a pandas DataFrame
results_df = pd.DataFrame({
    'Model': ['XGBoost'], # Add 'Linear Regression', 'Random Forest' if those models were trained
    'RMSE': [rmse_xgb], # Add rmse_lr, rmse_rf
    'MAE': [mae_xgb] # Add mae_lr, mae_rf
})

print("Model Performance Comparison:")
display(results_df)

## Chapter 6: Analysis of the Best Model

Based on the evaluation metrics, we select the best-performing model and conduct a deeper analysis. Since XGBoost generally performs well, we will focus on its analysis here.

### 6.1. Visualize Predictions vs. Actuals

In [ ]:
# Create a DataFrame for plotting
plot_df = pd.DataFrame({'Actual': y_val, 'Predicted': y_pred_xgb}, index=X_val.index)

# Sort by Date for a meaningful time series plot
plot_df = plot_df.sort_index()

plt.figure(figsize=(15, 6))
plt.plot(val_df['Date'], plot_df['Actual'], label='Actual Demand', alpha=0.7)
plt.plot(val_df['Date'], plot_df['Predicted'], label='Predicted Demand', alpha=0.7)
plt.title('XGBoost Predictions vs. Actuals Over Time')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 6.2. Feature Importance Analysis

In [ ]:
# Get feature importances from the trained XGBoost model
feature_importances = xgb_model.feature_importances_

# Create a DataFrame of feature importances
feature_importance_df = pd.DataFrame({'feature': X_train.columns, 'importance': feature_importances})

# Sort features by importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Display top 20 features
print("\nTop 20 Feature Importances (XGBoost):")
display(feature_importance_df.head(20))

# Plot feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importance_df.head(20))
plt.title('Top 20 Feature Importances (XGBoost)')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()